In [1]:
import os
import pandas as pd
import tensorflow as tf
from absl import logging
from modules import components, pipeline
from modules.utils import merge_dataset
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

c:\Users\zis\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
merge_dataset("data/movies/movies.csv", "data/ratings/ratings.csv")

'Merge dataset success'

In [3]:
PIPELINE_NAME = "cbf_pipeline"

# pipeline inputs
DATA_ROOT = "data/merge"
TRANSFORM_MODULE_FILE = "modules/cbf_transform.py"
TUNER_MODULE_FILE = "modules/cbf_tuner.py"
TRAINER_MODULE_FILE = "modules/cbf_trainer.py"

# pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")


In [4]:
components = components.init_components(
    data_dir=DATA_ROOT,
    transform_module=TRANSFORM_MODULE_FILE,
    tuner_module=TUNER_MODULE_FILE,
    trainer_module=TRAINER_MODULE_FILE,
    train_steps=1000,
    eval_steps=500,
    serving_model_dir=os.path.join(
        serving_model_dir, "cbf_model"),
    epochs=5,
)

pipeline = pipeline.init_pipeline(
    pipeline_root=pipeline_root,
    pipeline_name=PIPELINE_NAME,
    metadata_path=metadata_path,
    components=components,
)

BeamDagRunner().run(pipeline)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: outputs\cbf_pipeline\Transform\transform_graph\19\.temp_path\tftransform_tmp\e37ce6e7fd364bc38133514a529dab7d\assets


INFO:tensorflow:Assets written to: outputs\cbf_pipeline\Transform\transform_graph\19\.temp_path\tftransform_tmp\e37ce6e7fd364bc38133514a529dab7d\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
ERROR:absl:Execution 21 failed.


SyntaxError: invalid syntax [while running 'Run[Tuner]'] (cbf_tuner.py, line 79)